<a href="https://colab.research.google.com/github/Northern-Necker/local-ragv2/blob/main/kohya_ss_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === FINAL, PATCHED, AND ROBUST SETUP CELL ===

# 1. Login and Mount Drive
from google.colab import drive
from huggingface_hub import notebook_login
import os

print("➡️ Mounting Google Drive...")
drive.mount('/content/drive')

print("\n➡️ Please log into Hugging Face...")
notebook_login()

# 2. Clean up any previous installation attempts
print("\n➡️ Cleaning up old Kohya SS directory if it exists...")
if os.path.exists('/content/kohya_ss'):
    !rm -rf /content/kohya_ss

# 3. Install Correct, Compatible Libraries
print("\n➡️ Installing compatible libraries (PyTorch, xformers)...")
# Using --force-reinstall to ensure correct versions are active
!pip install --force-reinstall torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install -q xformers==0.0.22.post7

# 4. Clone Kohya SS GUI
print("\n➡️ Cloning the Kohya SS repository...")
%cd /content
!git clone https://github.com/camenduru/kohya_ss
%cd kohya_ss

# 5. *** CRITICAL PATCH ***: Modify the Python version requirement
print("\n🔧 Patching Python version constraint...")
!sed -i 's/python_version~="3.10"/python_version>="3.10"/' /content/kohya_ss/pyproject.toml
!sed -i 's/torch~=2.0.1/torch~=2.1.0/' /content/kohya_ss/pyproject.toml


# 6. Install Kohya SS and its dependencies
print("\n➡️ Installing Kohya SS GUI and its dependencies...")
!pip install -q .

# 7. Manually install/upgrade any potentially problematic packages
print("\n➡️ Force-installing key dependencies to prevent errors...")
!pip install -q lion-pytorch easygui "gradio==3.50.2"

# 8. Announce Folder Structure
print("\n✅ Setup complete. Please ensure your images are in the following folders:")
print("   - Instance images in: /content/drive/My Drive/kohya_ss/train/instance_images/10_ohwx teta woman")
print("   - Class images in:    /content/drive/My Drive/kohya_ss/train/class_images/1_woman")

# 9. Launch the GUI
print("\n🚀 Launching Kohya SS GUI... Please wait for the .gradio.live link.")
!python kohya_gui.py --share --headless

➡️ Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

➡️ Please log into Hugging Face...



➡️ Installing compatible libraries...
Looking in indexes: https://download.pytorch.org/whl/cu118

➡️ Installing Kohya SS GUI and manually installing dependencies...
/content
fatal: destination path 'kohya_ss' already exists and is not an empty directory.
/content/kohya_ss
Already up to date.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Package 'library' requires a different Python: 3.11.13 not in '<3.11,>=3.9'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 2.3 MB/s eta 0:00:00

➡️ Creating correct folder structure in Google Drive...

✅ Setup complete. Please ensure your images are in the following folders:
   - Instance images in: /content/drive/My Drive/kohya_ss/train/instance_images/10_ohwx teta woman
   - Class images in:    /content/drive/My Drive/kohya_ss/train/class_images/1_woman

🚀 Launching Kohya SS GUI... Please wait for the .gradio.live link.
12:45:30-467606 INFO     h

In [ ]:
#@title Convert Safetensors to Diffusers
from_safetensors_url = '' #@param {type:"string"}
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/v0.17.1/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget {from_safetensors_url} -O /content/model.safetensors
!python3 convert_original_stable_diffusion_to_diffusers.py --half --from_safetensors --checkpoint_path model.safetensors --dump_path /content/model

In [ ]:
#@title Push to HF.co

import os
from huggingface_hub import create_repo, upload_folder

hf_token = 'HF_WRITE_TOKEN' #@param {type:"string"}
repo_id = 'username/reponame' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
model_path = '/content/train/model' #@param {type:"string"}
upload_folder(folder_path=f'{model_path}', path_in_repo='', repo_id=repo_id, commit_message=commit_message, token=hf_token)

In [ ]:
#@title Push to DagsHub.com

!pip -q install dagshub
from dagshub.upload import Repo, create_repo

repo_id = 'reponame' #@param {type:"string"}
org_name = 'orgname' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(f"{repo_id}", org_name=f"{org_name}")
repo = Repo(f"{org_name}", f"{repo_id}")
model_path = '/content/train/model' #@param {type:"string"}
repo.upload("/content/models", remote_path="data", commit_message=f"{commit_message}", force=True)